In [1]:
import itertools

instructions = []
with open('day_22_test6_input') as f:
    for line in f.readlines():
        is_on = line.startswith('on')
        line = line.strip()[line.index('x'):]
        ranges = []
        range_strs = line.split(',')
        for i, range_str in enumerate(range_strs):
            range_str = range_str[2:]
            minstr, _, maxstr = range_str.partition('..')
            ranges.append((int(minstr), int(maxstr)))
        instructions.append((is_on, ranges))
print(instructions)

[(True, [(10, 12), (10, 12), (10, 12)]), (True, [(11, 13), (11, 13), (11, 13)]), (False, [(9, 11), (9, 11), (9, 11)]), (True, [(10, 10), (10, 10), (10, 10)]), (True, [(-20, 26), (-36, 17), (-47, 7)]), (True, [(-20, 33), (-21, 23), (-26, 28)]), (True, [(-22, 28), (-29, 23), (-38, 16)]), (True, [(-46, 7), (-6, 46), (-50, -1)]), (True, [(-49, 1), (-3, 46), (-24, 28)]), (True, [(2, 47), (-22, 22), (-23, 27)]), (True, [(-27, 23), (-28, 26), (-21, 29)]), (True, [(-39, 5), (-6, 47), (-3, 44)]), (True, [(-30, 21), (-8, 43), (-13, 34)]), (True, [(-22, 26), (-27, 20), (-29, 19)]), (False, [(-48, -32), (26, 41), (-47, -37)]), (True, [(-12, 35), (6, 50), (-50, -2)]), (False, [(-48, -32), (-32, -16), (-15, -5)]), (True, [(-18, 26), (-33, 15), (-7, 46)]), (False, [(-40, -22), (-38, -28), (23, 41)]), (True, [(-16, 35), (-41, 10), (-47, 6)]), (False, [(-32, -23), (11, 30), (-14, 3)]), (True, [(-49, -5), (-3, 45), (-29, 18)]), (False, [(18, 30), (-20, -8), (-3, 13)]), (True, [(-41, 9), (-7, 43), (-33, 

In [2]:
def build_grid():
    grid = []
    for x in range(-50, 51):
        plane = []
        for y in range(-50, 51):
            row = []
            for z in range(-50, 51):
                row.append(False)
            plane.append(row)
        grid.append(plane)
    return grid

In [5]:
minv = -50
maxv = 50

def patch_instruction(ins):
    patched = []
    for rng in ins:
        if rng[0] < minv:
            nmin = minv
        elif rng[0] > maxv:
            return None
        else:
            nmin = rng[0]
        if rng[1] < minv:
            return None
        elif rng[1] > maxv:
            nmax = maxv
        else:
            nmax = rng[1]
        patched.append((nmin, nmax))
    return patched

def score_grid(grid):
    score = 0
    for plane in grid:
        for row in plane:
            for val in row:
                if val:
                    score += 1
    return score
                    
grid = build_grid()
off = abs(minv)
for instruction in instructions:
    is_on, rng = instruction
    patched = patch_instruction(rng)
    if patched is None:
        continue
    # print(patched)
    for x in range(patched[0][0], patched[0][1] + 1):
        for y in range(patched[1][0], patched[1][1] + 1):
            for z in range(patched[2][0], patched[2][1] + 1):
                grid[x + off][y + off][z + off] = is_on
    print(score_grid(grid))


27
46
38
39
139629
210918
225476
328328
387734
420416
436132
478727
494759
494804
492164
534936
534936
567192
567150
592167
588567
592902
590029
590784
606951
626543
632603
633339
633229
654444
654192
654388
654252
664808
665269
665647
665647
700985
703864
726114
727074
744753


In [7]:
ndims = 3

class Point(object):
    
    def __init__(self, vals):
        self.vals = vals
        
    def __repr__(self):
        return ','.join([str(val) for val in self.vals])

class Square(object):
    
    def __init__(self, ranges):
        self.start = Point([r[0] for r in ranges])
        self.end = Point([r[1] for r in ranges])
        
    def key(self):
        vals = []
        for v in self.start.vals:
            vals.append(v)
        for v in self.end.vals:
            vals.append(v)
        return tuple(vals)
        
    def __repr__(self):
        return f'{{{self.start}->{self.end}}}'
    
s1 = Square([(0, 5), (13, 20), (0, 6)])
s2 = Square([(3, 10), (10, 15), (0, 6)])

# Returns true if pot is contained in exists
def is_in(pot, exists):
    for dim in range(ndims):
        if pot.start.vals[dim] < exists.start.vals[dim] or pot.end.vals[dim] > exists.end.vals[dim]:
            return False
    return True

def try_append(pot, existing, l):
    for exists in existing:
        if is_in(pot, exists):
            l.append(pot)
            return

def disjoint(s1, s2):
    for dim in range(ndims):
        if s1.end.vals[dim] < s2.start.vals[dim]:
            return True
        if s2.end.vals[dim] < s1.start.vals[dim]:
            return True
    return False
        
def non_matching_dims(s1, s2):
    non_matching = []
    for dim in range(ndims):
        if s1.start.vals[dim] != s2.start.vals[dim] or s1.end.vals[dim] != s2.end.vals[dim]:
            non_matching.append(dim)
    return non_matching
        
def is_adjacent(s1, s2, dim):
    return s1.end.vals[dim] == s2.start.vals[dim] or s2.end.vals[dim] == s1.start.vals[dim]

def merge_on_dim(s1, s2, dim):
    if s1.end.vals[dim] == s2.start.vals[dim]:
        merged_range = (s1.start.vals[dim], s2.end.vals[dim])
    elif s2.end.vals[dim] == s1.start.vals[dim]:
        merged_range = (s2.start.vals[dim], s1.end.vals[dim])
    else:
        raise Exception('Logic error')
    newranges = []
    for d in range(ndims):
        if d == dim:
            newranges.append(merged_range)
        else:
            newranges.append((s1.start.vals[d], s1.end.vals[d]))
    return Square(newranges)
    
def dedupe(squares):
    deduped = []
    myset = set()
    for square in squares:
        key = square.key()
        if key not in myset:
            deduped.append(square)
            myset.add(key)
    return deduped
    
def merge(squares):
    print("MERGE")
    at_least_one_merged = True
    while at_least_one_merged:
        newsquares = []
        at_least_one_merged = False
        matched_index = None
        for i, s1 in enumerate(squares):
            if matched_index is not None and matched_index != i:
                newsquares.append(s1)
            elif matched_index is None:
                for j, s2 in enumerate(squares[i+1:]):
                    nm_dims = non_matching_dims(s1, s2)
                    if len(nm_dims) == 1 and is_adjacent(s1, s2, nm_dims[0]):
                        at_least_one_merged = True
                        matched_index = j + i + 1
                        merged = merge_on_dim(s1, s2, nm_dims[0])
                        newsquares.append(merged)
                        break
            if not at_least_one_merged:
                newsquares.append(s1)
        squares = newsquares
    return squares
    
def intersect(s1, s2):
    if disjoint(s1, s2):
        return None
    sortvals = []
    for dim in range(ndims):
        sortvals.append(sorted((s1.start.vals[dim], s1.end.vals[dim] + 1, s2.start.vals[dim], s2.end.vals[dim] + 1)))
    squares = []
    
    bits = [(0, 1), (1, 2), (2, 3)]
    possibilities = itertools.product(bits, repeat=3)
    exists = [s1, s2]
    for possibility in possibilities:
        subranges = []
        for dim in range(ndims):
            sr_start = sortvals[dim][possibility[dim][0]]
            sr_end = sortvals[dim][possibility[dim][1]]
            if sr_start == sr_end:
                continue
            subrange = (sr_start, sr_end - 1)
            subranges.append(subrange)
        if len(subranges) != ndims:
            continue
        new_square = Square(subranges)
        try_append(new_square, exists, squares)
    
    return dedupe(squares)
    
def volume(s):
    if s is None:
        return 0
    mults = []
    for dim in range(ndims):
        mults.append(abs(s.end.vals[dim] - s.start.vals[dim]) + 1)
    result = 1
    for mult in mults:
        result *= mult
    return result
    
def crop(big, small):
    ranges = []
    for dim in range(ndims):
        biggest_start = max(big.start.vals[dim], small.start.vals[dim])
        smallest_end = min(big.end.vals[dim], small.end.vals[dim])
        if biggest_start > smallest_end:
            return None
        ranges.append((biggest_start, smallest_end))
    return Square(ranges)
     
def run():
    count_on = 0
    for i, instruction in enumerate(instructions):
        is_on, rang = instruction
        # print(f'Instruction {instruction}')
        if is_on:
            new_ranges = [Square(rang)]
            already_existed = []
            for prev in instructions[:i]:
                # print(f'  Factoring {prev}')
                prev_is_on, prev_range = prev
                prev_square = Square(prev_range)
                if prev_is_on:
                    new_new_ranges = []
                    for sq in new_ranges:
                        # print(f'  Considering region {sq}')
                        if disjoint(sq, prev_square):
                            # unrelated, keep
                            # print(f'  Keeping entire region {sq}')
                            new_new_ranges.append(sq)
                            continue
                        if is_in(sq, prev_square):
                            # Fully enveloped, drop
                            # print(f'  Moving fully enveloped region {sq}')
                            already_existed.append(sq)
                            continue
                        # print(f'  Intersecting {sq} and {prev_square}')
                        inter_squares = intersect(sq, prev_square)
                        for i, a in enumerate(inter_squares):
                            for b in inter_squares[i+1:]:
                                if not disjoint(a, b):
                                    print(f'  NON DISJOINT: {a} {b}')
                        #print(f'  intersect={inter_squares}')
                        exist_squares = [s for s in inter_squares if is_in(s, prev_square) and is_in(s, sq)]
                        # print(f'  exist={exist_squares}')
                        # exist_squares = merge(exist_squares)
                        for exist_square in exist_squares:
                            # print(f'  Moving overlapped subregion {exist_square}')
                            already_existed.append(exist_square)
                        inter_squares = [s for s in inter_squares if not is_in(s, prev_square) and is_in(s, sq)]
                        # inter_squares = merge(inter_squares)
                        for inter_square in inter_squares:
                            # print(f'  Keeping non-overlapped subregion {inter_square}')
                            new_new_ranges.append(inter_square)
                    new_ranges = new_new_ranges
                else:
                    new_existing_ranges = []
                    for sq in already_existed:
                        if disjoint(sq, prev_square):
                            # unrelated, keep existing
                            new_existing_ranges.append(sq)
                            continue
                        if is_in(sq, prev_square):
                            # Fully envoloped, existing moves to new
                            new_ranges.append(prev_square)
                            continue
                        inter_squares = intersect(sq, prev_square)
                        for i, a in enumerate(inter_squares):
                            for b in inter_squares[i+1:]:
                                if not disjoint(a, b):
                                    print(f'  NON DISJOINT: {a} {b}')
                        new_squares = [s for s in inter_squares if is_in(s, prev_square) and is_in(s, sq)]
                        # new_squares = merge(new_squares)
                        for new_square in new_squares:
                            new_ranges.append(new_square)
                        inter_squares = [s for s in inter_squares if not is_in(s, prev_square) and is_in(s, sq)]
                        # inter_squares = merge(inter_squares)
                        for inter_square in inter_squares:
                            new_existing_ranges.append(inter_square)
                    already_existed = new_existing_ranges          
            total_new = 0
            cropped = [crop(td, Square(rang)) for td in new_ranges]
            cropped = [c for c in cropped if c is not None]
            cropped = dedupe(cropped)
            for new_range in cropped:
                total_new += volume(new_range)
            count_on += total_new
            print(f'The instruction {instruction} added {total_new} new squares {count_on}')
        else:
            # print(f'Instruction: {instruction}')
            to_delete = []
            already_removed = [Square(rang)]
            # print(f'  Starting with {Square(rang)} in already removed')
            for prev in instructions[:i]:
                # print(f'  Considering instruction {prev}')
                prev_is_on, prev_range = prev
                prev_square = Square(prev_range)
                if prev_is_on:
                    new_already_removed = []
                    for sq in already_removed:
                        # print(f'  Considering region {sq}')
                        if disjoint(sq, prev_square):
                            # unrelated, keep
                            # print(f'  Keeping all of {sq} already removed')
                            new_already_removed.append(sq)
                            continue
                        if is_in(sq, prev_square):
                            # Fully enveloped, drop
                            # print(f'  Moving fully all of {sq} to to_delete')
                            to_delete.append(sq)
                            continue
                        inter_squares = intersect(sq, prev_square)
                        newadd_squares = [s for s in inter_squares if is_in(s, prev_square) and is_in(s, sq)]
                        # newadd_squares = merge(newadd_squares)
                        for newadd_square in newadd_squares:
                            # print(f'  Moving partial {newadd_square} to to_delete')
                            to_delete.append(newadd_square)
                        inter_squares = [s for s in inter_squares if not is_in(s, prev_square) and is_in(s, sq)]
                        # inter_squares = merge(inter_squares)
                        for inter_square in inter_squares:
                            # print(f'  Keeping partial {inter_square} to already removed')
                            new_already_removed.append(inter_square)
                    already_removed = new_already_removed
                else:
                    new_to_delete = []
                    for sq in to_delete:
                        # print(f'  Considering region {sq}')
                        if disjoint(sq, prev_square):
                            # unrelated, keep existing
                            # print(f'  Keeping all of {sq} in to_delete')
                            new_to_delete.append(sq)
                            continue
                        if is_in(sq, prev_square):
                            # Fully envoloped, existing moves to new
                            # print(f'  Moving fully all of {sq} to already_removed')
                            already_removed.append(prev_square)
                            continue
                        inter_squares = intersect(sq, prev_square)
                        prevdel_squares = [s for s in inter_squares if is_in(s, prev_square) and is_in(s, sq)]
                        # prevdel_squares = merge(prevdel_squares)
                        for prevdel_square in prevdel_squares:
                            # print(f'  Moving partial {prevdel_square} to already_removed')
                            already_removed.append(prevdel_square)
                        inter_squares = [s for s in inter_squares if not is_in(s, prev_square) and is_in(s, sq)]
                        # inter_squares = merge(inter_squares)
                        for inter_square in inter_squares:
                            # print(f'  Keeping partial {inter_square} in to_delete')
                            new_to_delete.append(inter_square)
                    to_delete = new_to_delete
            total_delete = 0
            cropped = [crop(td, Square(rang)) for td in to_delete]
            cropped = [c for c in cropped if c is not None]
            cropped = dedupe(cropped)
            for delete_range in cropped:
                total_delete += volume(delete_range)
            count_on -= total_delete
            print(f'The instruction {instruction} removed {total_delete} new squares {count_on}')
    print(count_on)

run()
# s1 = Square([(0, 2), (3, 4), (0, 0)])
# s2 = Square([(0, 5), (1, 8), (0, 0)])
# sqs = intersect(s1, s2)
# for sq in sqs:
#     print(f'{sq} {is_in(sq, s1)} {is_in(sq, s2)}')


The instruction (True, [(10, 12), (10, 12), (10, 12)]) added 27 new squares 27
The instruction (True, [(11, 13), (11, 13), (11, 13)]) added 19 new squares 46
The instruction (False, [(9, 11), (9, 11), (9, 11)]) removed 8 new squares 38
The instruction (True, [(10, 10), (10, 10), (10, 10)]) added 1 new squares 39
The instruction (True, [(-20, 26), (-36, 17), (-47, 7)]) added 139590 new squares 139629
The instruction (True, [(-20, 33), (-21, 23), (-26, 28)]) added 71289 new squares 210918
The instruction (True, [(-22, 28), (-29, 23), (-38, 16)]) added 14558 new squares 225476
The instruction (True, [(-46, 7), (-6, 46), (-50, -1)]) added 102852 new squares 328328
The instruction (True, [(-49, 1), (-3, 46), (-24, 28)]) added 59406 new squares 387734
The instruction (True, [(2, 47), (-22, 22), (-23, 27)]) added 32682 new squares 420416
The instruction (True, [(-27, 23), (-28, 26), (-21, 29)]) added 15716 new squares 436132
The instruction (True, [(-39, 5), (-6, 47), (-3, 44)]) added 42595 n